In [5]:
import os, numpy as np, pandas as pd
try:
    from gurobipy import Model, GRB, quicksum
except Exception as e:
    raise RuntimeError("Please install and license Gurobi (gurobipy) before running this cell.") from e

# Paths
BASE_DIR = os.path.expanduser("~/Downloads/IEOR4004EProject1")
PATH_REG = os.path.join(BASE_DIR, "child_c_care_regulated.csv") if False else os.path.join(BASE_DIR, "child_care_regulated.csv")
PATH_POP = os.path.join(BASE_DIR, "population.csv")
PATH_INC = os.path.join(BASE_DIR, "avg_individual_income.csv")
PATH_EMP = os.path.join(BASE_DIR, "employment_rate.csv")

# Assumption (idealistic): linear expansion cost per slot (optimistic)
EXPANSION_COST_PER_SLOT = 250.0
EQUIP_COST_0_5_PER_SLOT = 100.0  # equipment only for 0–5 (we use '-5' as proxy)
NEW_COST   = {"S": 65000.0, "M": 95000.0, "L": 115000.0}
NEW_TOTAL  = {"S": 100, "M": 200, "L": 400}
NEW_0_5_MAX = {"S": 50,  "M": 100, "L": 200}
NEW_5_12_MAX = {k: NEW_TOTAL[k] - NEW_0_5_MAX[k] for k in NEW_TOTAL}

# Data loading
reg = pd.read_csv(PATH_REG)
pop = pd.read_csv(PATH_POP)
inc = pd.read_csv(PATH_INC)
emp = pd.read_csv(PATH_EMP)

# ZIP rule: take only the first 5 digits
def _zip_trunc(val):
    s = ''.join(ch for ch in str(val) if ch.isdigit())
    return np.nan if not s else int(s[:5])

# Create a 'zipcode' column (do NOT rename your raw columns; just add a derived one for joins)
reg["zipcode"] = reg["zip_code"].apply(_zip_trunc)
pop["zipcode"] = pop["zipcode"].apply(_zip_trunc)
inc["zipcode"] = inc["ZIP code"].apply(_zip_trunc)        # <-- income file uses 'ZIP code'
emp["zipcode"] = emp["zipcode"].apply(_zip_trunc)

for d in (reg, pop, inc, emp):
    d.dropna(subset=["zipcode"], inplace=True)
    d["zipcode"] = d["zipcode"].astype(int)

# Aggregate existing capacity by ZIP 
reg["cap_0_5"]  = reg[["infant_capacity","toddler_capacity","preschool_capacity"]].sum(axis=1)
reg["cap_5_12"] = reg[["school_age_capacity"]].sum(axis=1)

by_zip = (reg.groupby("zipcode")
            .agg(cap0_5=("cap_0_5","sum"),
                 cap5_12=("cap_5_12","sum"),
                 total_cap=("total_capacity","sum"),
                 n_facilities=("facility_id","count"))
            .reset_index())

# Build ZIP table
# NOTE: Using '-5' (under-5) as proxy for "0–5" from policy
df = (by_zip
      .merge(pop[["zipcode","-5","5-9","10-14"]], on="zipcode", how="left")
      .merge(inc[["zipcode","average income"]], on="zipcode", how="left")     # <-- income column is 'average income'
      .merge(emp[["zipcode","employment rate"]], on="zipcode", how="left"))

df[["cap0_5","cap5_12","total_cap"]] = df[["cap0_5","cap5_12","total_cap"]].fillna(0.0)
df["n_facilities"] = df["n_facilities"].fillna(0).astype(int)

# Population pieces
df["pop_u5"]     = df["-5"].fillna(0.0)      # under-5 (proxy for 0–5)
df["pop_5_9"]    = df["5-9"].fillna(0.0)
df["pop_10_14"]  = df["10-14"].fillna(0.0)
df["pop_0_12"]   = df["pop_u5"] + df["pop_5_9"] + 0.6*df["pop_10_14"]

# Socio-econ for high-demand rule (use exact column names with spaces)
df["employment rate"]   = df["employment rate"].fillna(0.0)
df["average income"]    = df["average income"].fillna(1e9)
df["is_high_demand"]    = ((df["employment rate"] >= 0.60) | (df["average income"] <= 60000)).astype(int)

# Thresholds (strictly above desert line)
df["threshold_total"]   = np.where(df["is_high_demand"]==1, 0.50*df["pop_0_12"], (1/3.0)*df["pop_0_12"])
df["threshold_total"]   = np.ceil(df["threshold_total"] + 1.0)
df["threshold_u5"]      = np.ceil((2.0/3.0)*df["pop_u5"])

# Expansion cap per ZIP
df["expansion_cap"]     = np.minimum(0.20*df["total_cap"], 500.0*df["n_facilities"]).clip(lower=0.0)

# Model builder
m = Model("IEOR4004E_Idealistic")

# New facility counts (integers)
yS = {z: m.addVar(vtype=GRB.INTEGER, lb=0, name=f"yS[{z}]") for z in df["zipcode"]}
yM = {z: m.addVar(vtype=GRB.INTEGER, lb=0, name=f"yM[{z}]") for z in df["zipcode"]}
yL = {z: m.addVar(vtype=GRB.INTEGER, lb=0, name=f"yL[{z}]") for z in df["zipcode"]}

# Age allocations (using u5 proxy for 0–5)
u_new_u5 = {z: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"u_new_u5[{z}]") for z in df["zipcode"]}
e_u5     = {z: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"e_u5[{z}]")     for z in df["zipcode"]}
e_5_12   = {z: m.addVar(vtype=GRB.CONTINUOUS, lb=0.0, name=f"e_5_12[{z}]")   for z in df["zipcode"]}

m.update()

def total_new(z):     return NEW_TOTAL["S"]*yS[z] + NEW_TOTAL["M"]*yM[z] + NEW_TOTAL["L"]*yL[z]
def max_new_u5(z):    return NEW_0_5_MAX["S"]*yS[z] + NEW_0_5_MAX["M"]*yM[z] + NEW_0_5_MAX["L"]*yL[z]
def max_new_5_12(z):  return NEW_5_12_MAX["S"]*yS[z] + NEW_5_12_MAX["M"]*yM[z] + NEW_5_12_MAX["L"]*yL[z]

# Constraints
for _, r in df.iterrows():
    z = int(r["zipcode"])

    total_u5_after   = r["cap0_5"]  + e_u5[z]  + u_new_u5[z]
    total_5_12_after = r["cap5_12"] + e_5_12[z] + (total_new(z) - u_new_u5[z])
    total_after      = total_u5_after + total_5_12_after

    # Coverage
    m.addConstr(total_after     >= r["threshold_total"], name=f"not_desert[{z}]")
    m.addConstr(total_u5_after  >= r["threshold_u5"],   name=f"u5_policy[{z}]")

    # Facility age caps for new builds
    m.addConstr(u_new_u5[z] <= max_new_u5(z),                   name=f"new_u5_cap[{z}]")
    m.addConstr((total_new(z) - u_new_u5[z]) <= max_new_5_12(z), name=f"new_5_12_cap[{z}]")

    # Expansion cap
    m.addConstr(e_u5[z] + e_5_12[z] <= r["expansion_cap"], name=f"exp_cap[{z}]")

# Objective (idealistic): build + equipment(0–5 only) + linear expansion
build_cost  = quicksum(NEW_COST["S"]*yS[z] + NEW_COST["M"]*yM[z] + NEW_COST["L"]*yL[z] for z in df["zipcode"])
equip_cost  = quicksum(EQUIP_COST_0_5_PER_SLOT * (u_new_u5[z] + e_u5[z]) for z in df["zipcode"])
expand_cost = quicksum(EXPANSION_COST_PER_SLOT * (e_u5[z] + e_5_12[z])   for z in df["zipcode"])

m.setObjective(build_cost + equip_cost + expand_cost, GRB.MINIMIZE)
m.Params.OutputFlag = 1
m.optimize()

# Solution save
rows = []
for _, r in df.iterrows():
    z = int(r["zipcode"])
    y_s, y_m, y_l = int(round(yS[z].X)), int(round(yM[z].X)), int(round(yL[z].X))
    new_total = NEW_TOTAL["S"]*y_s + NEW_TOTAL["M"]*y_m + NEW_TOTAL["L"]*y_l
    rows.append(dict(
        zipcode=z,
        is_high_demand=int(r["is_high_demand"]),
        pop_u5=float(r["pop_u5"]), pop_0_12=float(r["pop_0_12"]),
        threshold_total=float(r["threshold_total"]), threshold_u5=float(r["threshold_u5"]),
        existing_cap_total=float(r["total_cap"]),
        existing_cap_0_5=float(r["cap0_5"]),
        existing_cap_5_12=float(r["cap5_12"]),
        expansion_cap=float(r["expansion_cap"]),
        y_small=y_s, y_medium=y_m, y_large=y_l,
        u_new_u5=float(u_new_u5[z].X),
        e_u5=float(e_u5[z].X),
        e_5_12=float(e_5_12[z].X),
        new_total_slots=float(new_total),
        total_slots_after=float(r["total_cap"] + e_u5[z].X + e_5_12[z].X + new_total),
        total_u5_after=float(r["cap0_5"] + e_u5[z].X + u_new_u5[z].X),
    ))

sol_df = pd.DataFrame(rows).sort_values("zipcode")
OUT = os.path.join(BASE_DIR, "idealistic_solution_by_zip.csv")
sol_df.to_csv(OUT, index=False)
print("\n=== Objective (USD) ===", f"{m.objVal:,.2f}")
print("Wrote:", OUT)

Set parameter Username
Set parameter LicenseID to value 2711138
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads



GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information